<a href="https://colab.research.google.com/github/Pratt33/sentiment_analysis_on_social_media/blob/main/sentiment_analysis_on_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/sentiment140.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import pandas as pd
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='latin-1')
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
df.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1599999 non-null  int64 
 1   ids     1599999 non-null  int64 
 2   date    1599999 non-null  object
 3   flag    1599999 non-null  object
 4   user    1599999 non-null  object
 5   text    1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [8]:
df['target'].value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

In [9]:
df['target'].replace(4, 1, inplace=True)
df.sample(5)

,target,ids,date,flag,user,text
1508713,1,2174694054,Sun Jun 14 23:06:11 PDT 2009,NO_QUERY,KevalPrabhu,@dlnarayan if I understand the tweet right.. A...
1141817,1,1977230590,Sat May 30 20:26:34 PDT 2009,NO_QUERY,irfanth,Morning tweeple. Congrats to @didilaksana @kap...
1397122,1,2053922173,Sat Jun 06 06:05:19 PDT 2009,NO_QUERY,CharlieCharx,sick party last night dont remember much but i...
650319,0,2237530854,Fri Jun 19 05:57:25 PDT 2009,NO_QUERY,RobertCustons,"@SpencerFosberry Lol, my tweet was nothin to ..."
73470,0,1694525739,Mon May 04 03:01:23 PDT 2009,NO_QUERY,Katchik99,Late night bad dreams will this loneliness ev...


In [10]:
df.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [11]:
df.isna().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [12]:
df.drop(['ids', 'flag', 'user', 'date'], axis=1, inplace=True)
df.head()

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [15]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [16]:
pattern = re.compile('[^a-zA-Z]')

In [19]:
import nltk
nltk.download('stopwords')
english_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [20]:
port_stemmer = PorterStemmer()

def stemming(content):
    # Replace non-alphabetic characters with space
    stemmed_content = re.sub(pattern, ' ', content)
    # Convert text to lowercase
    stemmed_content = stemmed_content.lower()
    # Split text into words
    stemmed_content = stemmed_content.split()
    # Stem words and remove stopwords
    stemmed_content = [port_stemmer.stem(word) for word in stemmed_content if word not in english_stopwords]
    # Join words back into a single string
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [21]:
df['stemmed_content'] = df['text'].apply(stemming)

In [31]:
df.head()

,target,text,stemmed_content
0,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,0,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,0,@Kwesidei not the whole crew,kwesidei whole crew


In [32]:
df['stemmed_content']

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1599999, dtype: object

In [33]:
X = df['stemmed_content'].values
y = df['target'].values

In [36]:
!pip install scikit-learn

from sklearn.model_selection import train_test_split # Import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Use train_test_split

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [38]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [39]:
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.777709375


In [40]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [41]:
from google.colab import files
files.download('model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [43]:
pred = loaded_model.predict(X_test[3])
print(pred)

[0]


In [44]:
if pred == 1:
  print('Positive')
else:
  print('Negative')

Negative
